In [5]:
# %matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign

In [6]:
# date0='0709_1157'
# date1='0709_1557'
# date2='0709_1934'
# dates=[date0,date1,date2]
# plate = 13
date0='0701_1957'
date1='0701_2357'
date2='0702_0357'
date3='0702_0757'
date4='0702_1157'
date5='0702_1557'
date6='0702_1957'
date7='0702_2357'
plate=13
dates=[date0,date1,date2,date3,date4]

In [8]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [ ]:
tabs=[]
for date in dates:
    tabs.append(pd.read_csv(get_path(date,plate,True,extension='_full.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [ ]:
for i, date in enumerate(dates):
    tabs_labeled[i].to_csv(f'Data/graph_{date}_{plate}_full_labeled.csv')
    tabs[i].to_csv(f'Data/graph_{date}_{plate}_full.csv')

In [10]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(f'Data/graph_{date}_{plate}_full_labeled.csv',
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [11]:
nx_graphs=[]
poss=[]
for tab in tabs_labeled:
    nx_graph,pos=generate_nx_graph(tab,labeled=True)
    nx_graphs.append(nx_graph)
    poss.append(pos)

In [85]:
def orient(pixel_list,root_pos):
    if np.all(root_pos==pixel_list[0]):
        return(pixel_list)
    else:
        return list(reversed(pixel_list))
    
def from_nx_to_tab_matlab(nx_graph,pos):
    column_names = ["origin_label","end_label","origin_posx", "origin_posy", "end_posx","end_posy", "pixel_list"]
    tab = pd.DataFrame(columns=column_names)
    for edge in nx_graph.edges:
        origin_label=edge[0]
        end_label=edge[1]
        origin_posx=pos[origin_label][0]
        origin_posy=pos[origin_label][1]
        end_posx = pos[end_label][0]
        end_posy = pos[end_label][1]
        pixel_list=orient(nx_graph.get_edge_data(*edge)['pixel_list'],pos[origin_label])
        new_line=pd.DataFrame({"origin_label":[origin_label],"end_label" : [end_label], 
                               "origin_posx":[origin_posx],"origin_posy":[origin_posy], "end_posx" : [end_posx],"end_posy" : [end_posy],
                               "pixel_list" : [','.join([f'{pixel[0]},{pixel[1]}' for pixel in pixel_list])]})
        tab=tab.append(new_line,ignore_index=True)
    return(tab)

In [87]:
from_nx_to_tab_matlab(nx_graphs[0],poss[0])

,origin_label,end_label,origin_posx,origin_posy,end_posx,end_posy,pixel_list
0,1878,1909,19837,746,19967,1200,"19837,746,19837,747,19837,748,19837,749,19837,..."
1,1909,1853,19967,1200,19789,1165,"19967,1200,19966,1200,19965,1200,19964,1200,19..."
2,1909,1949,19967,1200,20108,1281,"19967,1200,19968,1200,19969,1201,19970,1201,19..."
3,1853,1614,19789,1165,19111,2162,"19789,1165,19788,1164,19787,1164,19786,1163,19..."
4,1853,1933,19789,1165,20046,1358,"19789,1165,19789,1166,19789,1167,19789,1168,19..."
...,...,...,...,...,...,...,...
2530,1791,1884,19621,37990,19852,38161,"19621,37990,19622,37990,19623,37990,19624,3799..."
2531,1791,1705,19621,37990,19370,39166,"19621,37990,19621,37991,19620,37992,19620,3799..."
2532,1765,1747,19560,42352,19534,42409,"19560,42352,19559,42353,19559,42354,19558,4235..."
2533,1747,1760,19534,42409,19554,42463,"19534,42409,19535,42409,19536,42410,19537,4241..."


In [12]:
nx_graph_clean=[]
for graph in nx_graphs:
    S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    len_connected=[len(nx_graph.nodes) for nx_graph in S]
    nx_graph_clean.append(S[np.argmax(len_connected)])

In [13]:
skeletons=[]
for nx_graph in nx_graph_clean:
    skeletons.append(generate_skeleton(nx_graph,dim=(20687, 45554)))

In [14]:
factor = 5
final_pictures = [compress_skeleton(skeletons[i],factor) for i in range(len(skeletons))]

In [15]:
connections_growth_pattern = [whole_movement_identification(nx_graph_clean[i],nx_graph_clean[i+1],poss[i],poss[i+1]) for i in range(len(dates)-1)]

0.0
0.14084507042253522
0.28169014084507044
0.4225352112676056
0.5633802816901409
0.704225352112676
0.8450704225352113
0.9859154929577465
0.0
0.13262599469496023
0.26525198938992045
0.3978779840848806
0.5305039787798409
0.6631299734748011
0.7957559681697612
0.9283819628647215
0.0
0.12224938875305623
0.24449877750611246
0.36674816625916873
0.4889975550122249
0.6112469437652812
0.7334963325183375
0.8557457212713936
0.9779951100244498
0.0
0.11467889908256881
0.22935779816513763
0.3440366972477064
0.45871559633027525
0.573394495412844
0.6880733944954128
0.8027522935779816
0.9174311926605505


In [16]:
connections = [c[0] for c in connections_growth_pattern]
growth_pattern = [c[1] for c in connections_growth_pattern]

In [ ]:
def pinpoint_anastomosis(nx_graph_tm1,nx_grapht,equ):
    anastomosis=[]
    origins=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    for node in nx_grapht.nodes:
        if nx_grapht.degree(node)>=3 and node in equ.keys() and equ[node] in tips:
            anastomosis.append(node)
            origins.append(equ[node])
    return(anastomosis,origins)

In [17]:
growing_tips=[[node for node in growth_pattern[i].keys() if len(growth_pattern[i][node])>=20] for i in range(len(growth_pattern))]

In [18]:
degree3_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)>=3] for nx_graph in nx_graph_clean]

In [19]:
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

NameError: name 't' is not defined

In [ ]:
anastomosiss=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i])[0] for i in range (len(dates)-1)]
origins=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i])[1] for i in range (len(dates)-1)]

In [ ]:
t=1
tp1=t+1

In [ ]:
plot_t_tp1(origins[t],anastomosiss[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [ ]:
plot_t_tp1(growing_tips[t],growing_tips[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [ ]:
t=3
tp1=t+1
plot_t_tp1([2180],[2180],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [ ]:
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

In [61]:
def make_growth_picture_per_tip(pixels_from_tip,pos,shape=(20700,45600),factor=10,max_growth=200,min_growth=10,per_tip=True):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    number_tips = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        x=min(round(pos[tip][0]/factor),shape[0]//factor-1)
        y=min(round(pos[tip][1]/factor),shape[1]//factor-1)
        if growth<=max_growth:
#             print(number_tips)
            if growth>=min_growth:
                number_tips[x,y]+=1
                final_picture[x,y]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    if per_tip:
        return(final_picture/(number_tips+(number_tips==0).astype(np.int)),number_tips)
    else:
        return(final_picture,number_tips)

In [62]:
final_pictures_growth = [np.log(make_growth_picture_per_tip(growth_pattern[i],poss[i],factor=500,max_growth=4000,per_tip=True,min_growth=0)[0]+1) for i in range (len(growth_pattern))]


In [66]:
images = []
for i,picture in enumerate(final_pictures_growth):
    fig = plt.figure(figsize=(14,12))
    ax = fig.add_subplot(111)
    ax.imshow(picture)
    plt.savefig(f'Data/video_test/growth_timestep_{i}.png')
    plt.close(fig)
    images.append(imageio.imread(f'Data/video_test/growth_timestep_{i}.png'))
imageio.mimsave('Data/video_test/movie_growth.gif', images,duration=1)

In [46]:

import cv2
pos_problem=[11241, 20621]
xbegin=pos_problem[0]-1500
ybegin=pos_problem[1]-1500
xend=pos_problem[0]+1500
yend=pos_problem[1]+1500
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])
kernel = np.ones((5,5),np.uint8)
skeletons_small_dilated=[cv2.dilate(skeleton.todense().astype(np.uint8),kernel,iterations = 1) for skeleton in skeletons_small]

In [58]:
for tp1 in [1,2,3,4]:
    plot_t_tp1(node_smalls[tp1],node_smalls[tp1],poss[tp1],poss[tp1],skeletons_small_dilated[tp1],skeletons_small_dilated[tp1],shift=(xbegin,ybegin),save=f'Data/video_test/network_timestep_{tp1}')

In [67]:
import imageio
images = []
for t in [1,2,3,4]:
    images.append(imageio.imread(f'Data/video_test/network_timestep_{t}.png'))
imageio.mimsave('Data/video_test/movie.gif', images,duration=1)

In [57]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1,relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save=''):
    fig = plt.figure(figsize=(14,12))
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none')
    ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none')
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    for node in node_list_t:
        t = ax.text((pos_t[node][1]-shift[1])//compress, (pos_t[node][0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
                    size=10,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            t = ax.text((pos_tp1[node][1]-shift[1])//compress, (pos_tp1[node][0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
                        size=10,
                        bbox=bbox_props2)
    if len(save)>=1:
        plt.savefig(save)
        plt.close(fig)
    else:
        plt.show()